In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy
cartopy.config['data_dir'] = './maps'
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as feat

# When plotting with Cartopy, a deprecation warning pops up 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# It throws a SettingWithCopyWarning false positive
pd.options.mode.chained_assignment = None  # default='warn'


### Import data frames: run the jupyter notebook or read the .csv files
We will continue working on the df_emodnet and df_ifremer created by the previous Jupyter notebook. We can either choose to run the Jupyter notebook from here, or read the latest .csv file created.

In [ ]:
switch="Jupyter" # "Jupyter" or "csv"

vardict ={'datevec':'TIME',
      'lat':'LATITUDE','lon':'LONGITUDE','dep':'DEPTH', 'pres':'PRES',
      'ph': 'PHPH','phf': 'PHPH_QC',
      'wmo': 'WMO', 'platf': 'PLATFORM_NAME_OCEANOPS'}

if switch == "Jupyter":
    print("run carbon data from ERDDAP")
    %run "./Carbon_data_from_ERDDAP.ipynb"

elif switch == "csv":
    emodnetfiles=glob.glob('*EMODNet*.csv')
    df_emodnet=pd.read_csv(max(emodnetfiles, key=os.path.getctime),dtype={vardict['wmo']:str})
    bgcargofiles=glob.glob('*BGCArgo*.csv')
    df_ifremer=pd.read_csv(max(bgcargofiles, key=os.path.getctime),dtype={vardict['wmo']:str})
    

else:
    print("no valid input provided")


# Create a single dataframe
Concatenate the dataframe, unify the series types, drop unnecessary information, and order in chronological order

In [ ]:
dataframes=[df_emodnet,df_ifremer]
df=pd.concat(dataframes)

# Create a python datetime object in order to order and manipulate within the notebook
df['dateobject']=pd.to_datetime(df[vardict['datevec']])
df.sort_values(by=['dateobject',vardict['pres']],inplace=True)

display(df)

From a quick look at the dataframe, some data series may be dropped. `ph_in_situ_total_adjusted` seems to contain many non-valid values. `pres_adjusted` seems to differ very little from `PRES`. We will check both variables: if no valid pH_adjusted values are available, and the difference between pressure and pressure_adjusted is very small (in the order of 1e-5 dbar) we can remove them from the dataframe. We will also drop `depth (m)`, since we only have values coming from EMODnet, but we have `PRES` values for all the data points.

In [ ]:
valid_adph=df.shape[0]-sum(df['ph_in_situ_total_adjusted']>999.)-sum(df['ph_in_situ_total_adjusted'].isnull())
print('There are', str(valid_adph),
      'valid values of ph_in_situ_total_adjusted')
if valid_adph == 0:
    df.drop(columns=['ph_in_situ_total_adjusted', 'ph_in_situ_total_adjusted_qc'], inplace=True)

print('The average offset between pressure an pressure_adjusted is', 
      str(np.mean(df[vardict['pres']]-df['pres_adjusted (decibar)'])), 
      'and the standard deviation is',
      str(np.std(df[vardict['pres']]-df['pres_adjusted (decibar)'])))

if((abs(np.mean(df[vardict['pres']]-df['pres_adjusted (decibar)'])) < 0.00001) &
  (abs(np.std(df[vardict['pres']]-df['pres_adjusted (decibar)']) < 0.00001))):
    df.drop(columns=['pres_adjusted (decibar)'], inplace=True)
    
df.drop(columns=['depth (m)'], inplace=True)

display(df)


## Plotting data: map and position-depth / time series-depth
Let's visualize the data we fetched. First we'll plot a world map, color-coded by data source, and a longitude-depth scatter plot to identify which cluster of measurements they correspond to.

In [ ]:
proj=ccrs.cartopy.crs.Miller()
plt.figure(dpi=200)
ax = plt.axes(projection=proj)

#maxlon=max(df_emodnet[vardict['lon']].max(), df_ifremer[vardict['lon']].max())
#maxlat=max(df_emodnet[vardict['lat']].max(), df_ifremer[vardict['lat']].max())
#minlon=min(df_emodnet[vardict['lon']].min(), df_ifremer[vardict['lon']].min())
#minlat=min(df_emodnet[vardict['lat']].min(), df_ifremer[vardict['lat']].min())

if ((df[vardict['lon']].min() > -175) & (df[vardict['lon']].max() < 175) & 
    (df[vardict['lat']].min() > -85) & (df[vardict['lat']].max() < 85)) :
    ax.set_extent([df[vardict['lon']].min()-5,df[vardict['lon']].max()+5,
                   df[vardict['lat']].max()+5,df[vardict['lat']].min()-5])

ax.stock_img()
ax.coastlines()

scattercolor=['b','r']
scattersize=[20,5]
counter=0

for source in df.SOURCE.unique():
    sc=ax.scatter(df[df.SOURCE==source][vardict['lon']],
        df[df.SOURCE==source][vardict['lat']],
        c=scattercolor[counter],s=scattersize[counter],
        label=source,
        transform=ccrs.PlateCarree())
    counter=counter+1

ax.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=[10,6])
gs = fig.add_gridspec(2, hspace=0.2)
axs = gs.subplots(sharex=True, sharey=True)

# Common labels (matplotlib 3.4 and higher)
#fig.supylabel('pressure (dbar)')
#fig.supxlabel('longitude')

counter=0
for source in df.SOURCE.unique():
    sc=axs[counter].scatter(df[df.SOURCE==source][vardict['lon']],
        df[df.SOURCE==source][vardict['pres']],
        c=df[df.SOURCE==source][vardict['ph']],s=7,
        vmax=df[vardict['ph']].max(),
        vmin=df[vardict['ph']].min())  
    
    axs[counter].title.set_text(source)
    
    counter=counter+1

axs[0].invert_yaxis()
plt.xticks(rotation=45);

# Colorbar in its own axis
cb=fig.colorbar(sc, ax=axs.ravel().tolist())
cb.set_label('pH')

plt.show()


In [ ]:
fig = plt.figure(figsize=[10,6])
gs = fig.add_gridspec(2, hspace=0.2)
axs = gs.subplots(sharex=True, sharey=True)

# Common labels (matplotlib 3.4 and higher)
#fig.supylabel('pressure (dbar)')
#fig.supxlabel('date')

counter=0
for source in df.SOURCE.unique():
    sc=axs[counter].scatter(df[df.SOURCE==source]['dateobject'],
        df[df.SOURCE==source][vardict['pres']],
        c=df[df.SOURCE==source][vardict['ph']],s=7,
        vmax=df[vardict['ph']].max(),
        vmin=df[vardict['ph']].min())  
    
    axs[counter].title.set_text(source)
    
    counter=counter+1

axs[0].invert_yaxis()
plt.xticks(rotation=45);

# Colorbar in its own axis
cb=fig.colorbar(sc, ax=axs.ravel().tolist())
cb.set_label('pH')

plt.show()


The source datasets partially overlap: EMODnet gathers marine data from various sources; it is very likely that BGC-Argo is one of them. So let's quickly check the metadata.

Here we will find how many data points do not have a WMO number associated, how many different platforms are in the dataset, grouped by platform type, and which devices they are.
BGC-Argo `platform_type` gives the same information as Ocean-OPS, so there is no need to query the API.

In [ ]:
# Number of data points without WMO code
print("There are", str(sum(df_emodnet[vardict['wmo']].isnull())), "datapoints without a WMO number associated in the EMODNet dataframe")
print("There are", str(sum(df_ifremer[vardict['wmo']].isnull())), "datapoints without a WMO number associated in the BGC-Argo dataframe")

# Retrieve device types
wmobytype=df.groupby([df.EP_PLATFORM_TYPE, df.SOURCE, df[vardict['wmo']],df.data_assembly_center], as_index=False).agg({vardict['ph']:['size','mean']})
display(wmobytype)

We see that the Profiling float data, though differing in length, come from the same individual profile, same data center, and per the plot above, same area and time. There are now two options: 

**a)** assume (safely) that the best and most updated measurements of Argo floats come from BGC-Argo (IFREMER) and drop the EMODnet values completely, 

**b)** proceed in the search for duplicates. 

We will save a csv file with the option a) and explore option b) as far as it's feasible.

In [ ]:
df_bgcargo=df.copy()
df_bgcargo.drop(columns='dateobject', inplace=True)
df_bgcargo=df_bgcargo.loc[~((df_bgcargo['EP_PLATFORM_TYPE'] == 'PF') &
                     (df_bgcargo['SOURCE'] == 'EMODnet pH profiles'))]

wmobytype=df_bgcargo.groupby(['EP_PLATFORM_TYPE', 'SOURCE', vardict['wmo'],'data_assembly_center'], as_index=False).agg({vardict['ph']:['size','mean']})
display(wmobytype)
display(df_bgcargo[-1-10:-1])

print("The resulting dataframe has dimensions", df_bgcargo.shape)

dateforfile=datetime.now().strftime("%Y%m%d")
df_bgcargo.to_csv(dateforfile+'_pH_data_ERDDAP_combined_PFonlyBGCArgoIFREMER.csv', index=False)

Let's try to identify how many exact duplicates there are in the dataframe by using the `duplicated` method
 drop them and plot (zooming in).

In [ ]:
isdup=df.duplicated(subset=[vardict['wmo'],vardict['lat'],vardict['lon'],'dateobject',vardict['pres'],vardict['ph']], 
                    keep=False)
print(str(sum(isdup)),"rows are full duplicates (pH values included)")

df.drop_duplicates(subset=[vardict['wmo'],vardict['lat'],vardict['lon'],'dateobject',vardict['pres'],vardict['ph']],keep="first", inplace=True, ignore_index=True)

fig, ax = plt.subplots(figsize=[12,6])
colors = {'BGC-Argo (IFREMER)':'red', 'EMODnet pH profiles':'blue'}
sizes={'BGC-Argo (IFREMER)':5,'EMODnet pH profiles':5}
ax.scatter(pd.to_datetime(df[vardict['datevec']]), df[vardict['pres']], c=df['SOURCE'].map(colors), s=df['SOURCE'].map(sizes))
plt.ylabel('pressure (dbar)')
ax.invert_yaxis();
plt.xticks(rotation=45);
plt.xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
plt.ylim(0,10)
plt.show()

Many duplicates seem to remain, in very close proximity. Let's check visually the data values from the profiling float; the duplicates may not be exact, but approximate.

In [ ]:
display(df.loc[df[vardict['wmo']]=='6902879',[vardict['wmo'],vardict['lat'],vardict['lon'],'dateobject',vardict['pres'],vardict['ph'],'SOURCE']])

It seems that the differences come down to the significant digits, so we will round the values, including the timestamps to the closest minute, and compare again. We will make a copy first of the dataframe (we might want to keep all the significant digits).

In [ ]:
df_unrounded=df.copy() # Preserve a copy of the original

df=df.round({vardict['ph']: 3, vardict['pres']: 0,vardict['lat']:5,vardict['lon']:5})
df.sort_values(by=['dateobject',vardict['pres']],inplace=True)

isdup=df.duplicated(subset=[vardict['wmo'],vardict['lat'],vardict['lon'],'dateobject',vardict['pres'],vardict['ph']], 
                    keep=False)
print(str(sum(isdup)),"rows are full duplicates after rounding")

df.drop_duplicates(subset=[vardict['wmo'],vardict['lat'],vardict['lon'],'dateobject',vardict['pres'],vardict['ph']],keep="first", inplace=True, ignore_index=True)

print("The dataframe without duplicates has the dimensions",df.shape,"vs",df_unrounded.shape)

fig, ax = plt.subplots(figsize=[12,6])
colors = {'BGC-Argo (IFREMER)':'red', 'EMODnet pH profiles':'blue'}
sizes={'BGC-Argo (IFREMER)':5,'EMODnet pH profiles':5}
ax.scatter(pd.to_datetime(df[vardict['datevec']]), df[vardict['pres']], c=df['SOURCE'].map(colors), s=df['SOURCE'].map(sizes))
plt.ylabel('pressure (dbar)')
ax.invert_yaxis();
plt.xticks(rotation=45);
plt.xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
plt.ylim(0,10)
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=False, figsize=[16,8])

colors = {'BGC-Argo (IFREMER)':'red', 'EMODnet pH profiles':'blue'}
sizes={'BGC-Argo (IFREMER)':5,'EMODnet pH profiles':5}

# Common labels (matplotlib 3.4 and higher)
#fig.supylabel('pressure (dbar)')
#fig.supxlabel('date')

axs[0,0].scatter(pd.to_datetime(df[vardict['datevec']]), df[vardict['pres']], c=df['SOURCE'].map(colors), s=df['SOURCE'].map(sizes))
axs[0,0].set_xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
axs[0,0].invert_yaxis()
axs[0,0].title.set_text('Rounded values, duplicates removed')

axs[1,0].scatter(pd.to_datetime(df[vardict['datevec']]), df[vardict['pres']], c=df['SOURCE'].map(colors), s=df['SOURCE'].map(sizes))
axs[1,0].set_xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
axs[1,0].set_ylim(0,10)
axs[1,0].invert_yaxis()
plt.setp(axs[1,0].get_xticklabels(), rotation=45, ha='right')

axs[0,1].scatter(pd.to_datetime(df_unrounded[vardict['datevec']]), df_unrounded[vardict['pres']], c=df_unrounded['SOURCE'].map(colors), s=df_unrounded['SOURCE'].map(sizes))
axs[0,1].set_xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
axs[0,1].invert_yaxis()
axs[0,1].title.set_text('Not-rounded, with duplicates')

axs[1,1].scatter(pd.to_datetime(df_unrounded[vardict['datevec']]), df_unrounded[vardict['pres']], c=df_unrounded['SOURCE'].map(colors), s=df_unrounded['SOURCE'].map(sizes))
axs[1,1].set_xlim(pd.to_datetime('2017-12-01'), pd.to_datetime('2018-01-01'))
axs[1,1].set_ylim(0,10)
axs[1,1].invert_yaxis()
plt.setp(axs[1,1].get_xticklabels(), rotation=45, ha='right')

plt.show()

So far, we have identified and eliminated duplicates, and we can export to a csv file, for later use.

In [ ]:
dateforfile=datetime.now().strftime("%Y%m%d")
df.to_csv(dateforfile+'_pH_data_ERDDAP_combined_removeduplicates.csv', index=False)